In [46]:
import os
import glob
import json
import polars as pl
from analysis import get_network_status

In [47]:
summaries = []
summaries.extend(glob.glob('../../results/aoi/*/summary.json'))
summaries.extend(glob.glob('../../results/aoi/*/*/summary.json'))

In [48]:
# pl.Config.set_tbl_hide_dataframe_shape(True)
status = []
for summary in summaries:
    with open(summary, 'r') as fin:
        net_status = get_network_status(json.load(fin))
    stats = pl.DataFrame(net_status).mean().to_dicts()[0]
    stats['rsrq'] = stats['rsrq'] / 10
    stats['rsrp'] = stats['rsrp'] / 10
    stats['rssi'] = stats['rssi'] / 10
    stats['experiment'] = '/'.join(os.path.split(summary)[0].split('\\'))
    status.append(stats)

df = pl.DataFrame(status)
df.sort(by='experiment')

df.write_csv('../../results/aoi/network_status.csv', float_precision=3)